In [ ]:
from tkinter import *
import tkinter as tk

import PIL

from tkinter import filedialog
master = Tk()
var = StringVar()
emo = StringVar()
fame = StringVar()
master.title('EMOTION DETECTION')
##master.overrideredirect(True)
master.geometry("{0}x{1}+0+0".format(master.winfo_screenwidth(), master.winfo_screenheight()))


topFrame = tk.Frame(master, bg = 'black', width = 1500, height=100, relief = 'raised') # , padx = 100, pady=100
topFrame.grid(row = 0, column = 0, columnspan = 3,  sticky="w")



LFrame = tk.Frame(master, width = 800, height = 900, relief = 'raised') # , padx = 100, pady=100
LFrame.grid(row = 2, column = 0,  sticky="nsew")

RFrame = tk.Frame(master, width = 700, height = 100, relief = 'raised') # , padx = 100, pady=100
RFrame.grid(row = 2, column = 1,  sticky="nsew")


def callback():
    import pyaudio
    import wave

    CHUNK = 1024 
    FORMAT = pyaudio.paInt16 #paInt8
    CHANNELS = 2 
    RATE = 44100 #sample rate
    RECORD_SECONDS = 4
    WAVE_OUTPUT_FILENAME = "output.wav"

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,

                frames_per_buffer=CHUNK) #buffer

    print("* recording")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data) # 2 bytes(16 bits) per channel

    print("* done recording")

    stream.stop_stream()
    stream.close()
    p.terminate()
    
 

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    
def findemo():
   
    import pandas as pd
    import librosa
    import numpy as np
    from keras.utils import np_utils
    from sklearn.preprocessing import LabelEncoder
    from PIL import Image, ImageTk
    from keras.models import load_model
    lmodel=load_model("Emotion_Voice_Detection_Model.h5")
    
    lb = LabelEncoder()
    lb.classes_ = np.load('classes.npy')
   
    X, sample_rate = librosa.load(master.filename, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
    
    featurelive = mfccs
    livedf2 = featurelive
    livedf2= pd.DataFrame(data=livedf2)
    livedf2 = livedf2.stack().to_frame().T 
    
    twodim= np.expand_dims(livedf2, axis=2)
    
    livepreds = lmodel.predict(twodim,batch_size=32,verbose=1)
    livepreds1=livepreds.argmax(axis=1)
    liveabc = livepreds1.astype(int).flatten()
    print(livepreds)
    livepredictions = (lb.inverse_transform((liveabc)))
    print(livepredictions[0])
    emo.set(livepredictions[0])  
        
def upload():
    
    global filename
    master.filename =  filedialog.askopenfilename(initialdir = "/home/user/main_project/",title = "Select file",filetypes = (("audio","*.wav"),("all files","*.*")))
    print(master.filename)
    global x
    x=master.filename
    print("x:")
    print(x)
    fdetail = tk.Label(LFrame, text="FILE NAME",font=("Arial",18))
    fname = tk.Label(LFrame, textvariable=x,font=("Arial",18))
    fdetail.place(x=200,y=210)
    fname.place(x=500,y=210)

    
def detectemotion():
    import pandas as pd
    import librosa
    import numpy as np
    from keras.utils import np_utils
    from sklearn.preprocessing import LabelEncoder
   
    from keras.models import load_model
    lmodel=load_model("Emotion_Voice_Detection_Model.h5")
    
    lb = LabelEncoder()
    lb.classes_ = np.load('classes.npy')
    
    X, sample_rate = librosa.load('output.wav', res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
    
    featurelive = mfccs
    livedf2 = featurelive
    livedf2= pd.DataFrame(data=livedf2)
    livedf2 = livedf2.stack().to_frame().T 
    
    twodim= np.expand_dims(livedf2, axis=2)
    
    livepreds = lmodel.predict(twodim,batch_size=32,verbose=1)
    livepreds1=livepreds.argmax(axis=1)
    liveabc = livepreds1.astype(int).flatten()
    print(livepreds)
    livepredictions = (lb.inverse_transform((liveabc)))
    print(livepredictions[0])
    var.set(livepredictions[0])
 
    
label1 = Label(LFrame, text="UPLOAD VOICE",font=("Arial",18))
b1 = Button(LFrame, text="BROWSE", command=upload)
label2 = Label(LFrame, text="DETECT EMOTION",font=("Arial",18))
b2 = Button(LFrame, text="DETECT", command=findemo)
labeld = Label(LFrame,text="DETECTED EMOTION",font=("Arial",18))
label3 = Label(LFrame,textvariable=emo,font=("Arial",18))



label4 = Label(LFrame, text="RECORD YOUR VOICE          ",font=("Arial",18))
b3 = Button(LFrame, text="RECORD", command=callback)
label5 = Label(LFrame, text="DETECT EMOTION        ",font=("Arial",18))
b4 = Button(LFrame, text="DETECT",command=detectemotion)
labeld1 = Label(LFrame,text="DETECTED EMOTION",font=("Arial",18))
label6 = Label(LFrame,textvariable=var,font=("Arial",18))
b5 = Button(LFrame, text="EXIT",command=master.destroy)

title = tk.Label(topFrame, text="EMOTION DETECTION",font=("Arial",38),fg="white",bg="black")
title.place(x=400,y=10)


label1.place(x=200,y=160)
b1.place(x=500,y=160)

label2.place(x=200,y=250)
b2.place(x=500,y=250)
labeld.place(x=200,y=290)
label3.place(x=500,y=290)


b3.place(x=500,y=440)
label4.place(x=200,y=440)
b4.place(x=500,y=480)
label5.place(x=200,y=480)
b5.place(x=500,y=600)
labeld1.place(x=200,y=520)
label6.place(x=500,y=520)


mainloop()


/home/user/main_project/Audioo/Test_audio/a02.wav
x:
/home/user/main_project/Audioo/Test_audio/a02.wav
1/1 [==============================] - 0s 228ms/step
[[3.25285587e-37 0.00000000e+00 0.00000000e+00 1.08377126e-29
  0.00000000e+00 9.99998212e-01 3.28040581e-23 5.52000390e-10
  1.77677759e-06 3.53654777e-08]]
male_angry
/home/user/main_project/Audioo/Test_audio/h09.wav
x:
/home/user/main_project/Audioo/Test_audio/h09.wav
1/1 [==============================] - 0s 261ms/step
[[4.7420530e-34 0.0000000e+00 0.0000000e+00 1.5200334e-35 3.5276728e-38
  1.7607232e-04 6.8674724e-21 3.1131334e-10 9.9982399e-01 2.0347628e-09]]
male_happy
/home/user/main_project/Audioo/Test_audio/sa14.wav
x:
/home/user/main_project/Audioo/Test_audio/sa14.wav
1/1 [==============================] - 0s 290ms/step
[[5.2340327e-37 0.0000000e+00 7.5734976e-36 8.2576100e-34 0.0000000e+00
  7.0950676e-08 3.1555166e-27 3.3541884e-13 2.7235287e-11 9.9999988e-01]]
male_sad
/home/user/main_project/Audioo/Test_audio/sa14.wa